In [107]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import iso8601
import datetime
from datetime import datetime, timedelta, date
import dateparser

# https://www.leboncoin.fr/recherche?category=43&text=ps5
# dans un dataframe pandas : title, price, date posted, city, postal code


def extract_leboncoin_data():
    domain_name = 'https://www.leboncoin.fr'
    route = "/recherche"
    query_param = "category=43&text=ps5"
    payload = {'category': '43', 'text': 'ps5'}
    headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}
    res = requests.get('https://www.leboncoin.fr/recherche', params=payload, headers=headers)
    soup = BeautifulSoup(res.text)

    #get <a> tag, root tag to all articles
    p_articles = soup.findAll("a", {"class":"_2-MzW _23lZh HlrAk _2k-6T"})

    #articles list to be later transformed into dataframe
    articles = []

#iterate through all html tag, i.e. all retrieved articles
    for article in p_articles:
        title = article.find("p", {"class":"_1MlU1 _1qZ_s"})['title']
        price = article.find("span", {"class":"_1C-CB"}).text.split()[0] if article.find("span", {"class":"_1C-CB"}) != None else 'NONE'

        #all the following info has the same class --> get all in table and then get them with index
        
        place_info = article.findAll("div", {"class":"_1UzWr"})[1].text.split()
        
        city = ''

#Le[0] mans[1] en [2] 12000[3]
        for i in range(0, len(place_info)-1):
            city += place_info[i] + ' '
        city.rstrip()
        
        postal_code = article.findAll("div", {"class":"_1UzWr"})[1].text.split()[len(place_info)-1]
        posting_date = article.findAll("div", {"class":"_1UzWr"})[2].text

        #divide date into date, hour and minute to handle the "aujourd'hui" and "hier" as date
        #date without time
        posting_date_truncated = posting_date.split()[0].rstrip(',')
        

        # get hour
        posting_hour = posting_date.split(',')[1].split(':')[0].strip()

        #get minute
        posting_minute = posting_date.split(',')[1].split(':')[1].strip()

        if(posting_date_truncated.lower() =='aujourd\'hui'):
            current_day = datetime.now() 
            current_day = current_day.replace(hour=int(posting_hour), minute=int(posting_minute)).replace(microsecond=0).replace(second=0)
            posting_date = current_day

        elif(posting_date_truncated.lower() =='hier'):
            current_day = datetime.now() - timedelta(days=1)
            current_day = current_day.replace(hour=int(posting_hour), minute=int(posting_minute)).replace(microsecond=0).replace(second=0)

            posting_date = current_day
            
        else: # 18 déc, 18:00 format
            posting_date = dateparser.parse(posting_date_truncated).date()
            posting_date = datetime.combine(date.today(), datetime.min.time())
            posting_date = posting_date.replace(hour=int(posting_hour), minute=int(posting_minute)).replace(microsecond=0).replace(second=0)


        # format the date into ISO 8601
        posting_date = posting_date.isoformat()

        dico = {
            "title": str(title),
            "price": str(price),
            "posting_date": posting_date,
            "city": str(city),
            "postal_code": str(postal_code)
        }

        articles.append(dico)

    # display(articles)
    
    df_leboncoin = pd.DataFrame(articles)
    return df_leboncoin

# END extract_leboncoin_data()

result = extract_leboncoin_data()
display(result)



,title,price,posting_date,city,postal_code
0,PlayStation 5 standard blu ray neuve,900,2020-12-18T00:44:00,Courbevoie,92400
1,Protections joystick ps5/ps4/xbox/switch,5,2020-12-18T00:30:00,Bretagne-de-Marsan,40280
2,PS5 exclusive,1,2020-12-18T00:30:00,Nancy,54000
3,PS5 neuve avec facture,750,2020-12-18T00:27:00,Paris,75008
4,Watch Dogs Légion PS4 + mise à jour PS5 incluse,30,2020-12-18T00:24:00,Châtillon-sur-Chalaronne,01400
5,Ps5 Digital neuf sous blister,800,2020-12-18T00:21:00,Paris,75116
6,Playstation 5 neuve jamais déballé,700,2020-12-18T00:20:00,Vernon,27200
7,Spider Man Miles Morales PS5,49,2020-12-18T00:18:00,Saint-Ouen-l'Aumône,95310
8,Console Sony PS4 Pro 1 To + manette + jeu,280,2020-12-18T00:17:00,Boissy-Saint-Léger,94470
9,PlayStation 5 édition avec lecteur Blu-ray,1,2020-12-18T00:15:00,Clermont-Ferrand,63000
